# 05 — Perturbation & Shock Experiments

Study how the market responds to:
1. Sudden fundamental value shocks (crash/boom)
2. Recovery dynamics and time to mean-reversion
3. Regime changes in strategy composition

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from market_abm import MarketModel, DEFAULT_PARAMS
from market_abm.visualization import plot_price_and_fundamental, plot_strategy_fractions

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## Experiment 1: Single crash (-15% fundamental shock at t=500)

In [ ]:
params = {**DEFAULT_PARAMS, 'steps': 2000, 'n_agents': 200, 'seed': 42}

model_crash = MarketModel(params)
model_crash.schedule_intervention(500, -0.15)  # 15% crash
results_crash = model_crash.run()
data_crash = results_crash.variables.MarketModel

# Baseline (no shock)
model_base = MarketModel(params)
results_base = model_base.run()
data_base = results_base.variables.MarketModel

print('Crash experiment complete.')

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Price comparison
axes[0].plot(data_base.index, data_base['price'], alpha=0.6, label='Baseline Price')
axes[0].plot(data_crash.index, data_crash['price'], alpha=0.8, label='Crash Price')
axes[0].plot(data_crash.index, data_crash['fundamental'], '--', alpha=0.5, label='Fundamental (crash)')
axes[0].axvline(500, color='red', linestyle=':', label='Shock at t=500')
axes[0].set_title('Price Response to -15% Fundamental Shock')
axes[0].legend()
axes[0].set_ylabel('Price')

# Strategy fractions after crash
plot_strategy_fractions(data_crash, ax=axes[1])
axes[1].axvline(500, color='red', linestyle=':', alpha=0.7)
axes[1].set_title('Strategy Composition (crash scenario)')

# Log returns
axes[2].plot(data_crash.index, data_crash['log_return'], linewidth=0.5, alpha=0.8)
axes[2].axvline(500, color='red', linestyle=':', label='Shock')
axes[2].set_title('Log Returns (crash scenario)')
axes[2].set_xlabel('Step')
axes[2].set_ylabel('Log Return')
axes[2].legend()

plt.tight_layout()
fig.savefig('../figures/crash_experiment.png', dpi=150, bbox_inches='tight')
plt.show()

## Experiment 2: Recovery time analysis

Measure how many steps it takes for the price-to-fundamental ratio to return within 5% of parity after shocks of different magnitudes.

In [ ]:
shock_sizes = [-0.05, -0.10, -0.15, -0.20, -0.30]
recovery_times = []

for shock in shock_sizes:
    params = {**DEFAULT_PARAMS, 'steps': 3000, 'n_agents': 200, 'seed': 42}
    model = MarketModel(params)
    model.schedule_intervention(500, shock)
    results = model.run()
    data = results.variables.MarketModel
    
    # Find recovery: price within 5% of fundamental after shock
    post_shock = data.loc[data.index >= 500]
    ratio = (post_shock['price'] / post_shock['fundamental']).values
    
    recovery_step = None
    for i in range(1, len(ratio)):
        if abs(ratio[i] - 1.0) < 0.05:
            recovery_step = i
            break
    
    recovery_times.append({
        'shock_pct': f'{shock*100:.0f}%',
        'recovery_steps': recovery_step if recovery_step else '>2500',
    })

pd.DataFrame(recovery_times)

## Experiment 3: Double shock (crash + boom)

In [ ]:
params = {**DEFAULT_PARAMS, 'steps': 3000, 'n_agents': 200, 'seed': 42}
model_double = MarketModel(params)
model_double.schedule_intervention(500, -0.15)   # crash
model_double.schedule_intervention(1500, 0.20)   # boom
results_double = model_double.run()
data_double = results_double.variables.MarketModel

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

plot_price_and_fundamental(data_double, ax=axes[0])
axes[0].axvline(500, color='red', linestyle=':', label='Crash (-15%)')
axes[0].axvline(1500, color='green', linestyle=':', label='Boom (+20%)')
axes[0].legend()
axes[0].set_title('Price Response to Sequential Shocks')

plot_strategy_fractions(data_double, ax=axes[1])
axes[1].axvline(500, color='red', linestyle=':', alpha=0.7)
axes[1].axvline(1500, color='green', linestyle=':', alpha=0.7)

plt.tight_layout()
fig.savefig('../figures/double_shock.png', dpi=150, bbox_inches='tight')
plt.show()

## Experiment 4: Regime change — high β vs low β

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

for col, beta_val in enumerate([0.5, 5.0]):
    params = {**DEFAULT_PARAMS, 'steps': 3000, 'n_agents': 200, 'seed': 42,
              'beta': beta_val}
    model = MarketModel(params)
    results = model.run()
    data = results.variables.MarketModel
    
    plot_price_and_fundamental(data, ax=axes[0, col])
    axes[0, col].set_title(f'β = {beta_val}')
    
    plot_strategy_fractions(data, ax=axes[1, col])

plt.suptitle('Low vs High Intensity of Choice', fontsize=13, fontweight='bold')
plt.tight_layout()
fig.savefig('../figures/regime_comparison.png', dpi=150, bbox_inches='tight')
plt.show()